In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv('D:/DATASET/STOCK_DATA/ADANIPORTS.csv')
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values('Date')
data.head()

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble,connect
0,2007-11-27,MUNDRAPORT,EQ,440.00,770.00,1050.00,770.0,959.0,962.90,984.72,27294366,"$2,687,719,053,785,000.00",NaN,9859619,0.3612,1.0
1,2007-11-28,MUNDRAPORT,EQ,962.90,984.00,990.00,874.0,885.0,893.90,941.38,4581338,"$431,276,530,164,999.00",NaN,1453278,0.3172,NaN
2,2007-11-29,MUNDRAPORT,EQ,893.90,909.00,914.75,841.0,887.0,884.20,888.09,5124121,"$455,065,846,264,999.00",NaN,1069678,0.2088,NaN
3,2007-11-30,MUNDRAPORT,EQ,884.20,890.00,958.00,890.0,929.0,921.55,929.17,4609762,"$428,325,662,830,000.00",NaN,1260913,0.2735,NaN
4,2007-12-03,MUNDRAPORT,EQ,921.55,939.75,995.00,922.0,980.0,969.30,965.65,2977470,"$287,519,974,300,000.00",NaN,816123,0.2741,NaN


In [4]:
data.Symbol.value_counts()

Symbol
ADANIPORTS    2299
MUNDRAPORT    1023
Name: count, dtype: int64

In [5]:
print(data.Date.max(),"----",data.Date.min())
data.shape

2021-04-30 00:00:00 ---- 2007-11-27 00:00:00


(3322, 16)

In [6]:
data.isnull().sum()

Date                     0
Symbol                   0
Series                   0
Prev Close               0
Open                     0
High                     0
Low                      0
Last                     0
Close                    0
VWAP                     0
Volume                   0
Turnover                 0
Trades                 866
Deliverable Volume       0
%Deliverble              0
connect               3321
dtype: int64

In [7]:
data['Return'] = data['Close'].pct_change()
volatility = data['Return'].std() * np.sqrt(252)  # Assuming 252 trading days in a year
print(f"Historical Volatility: {volatility:.2%}")

Historical Volatility: 48.21%


<head>
    <script src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js"></script>
</head>
<body>

# Binomial Option Pricing Model

It uses an iterative procedure, allowing for the specification of nodes, or points in time, during the time span between the valuation date and the option's expiration date
The binomial option pricing model is a technique used to value options by simulating possible paths the underlying asset's price could take over the option's life. It assumes the price of the underlying asset can only move up or down by a certain amount in each time, creating a "binomial tree" of possible price movements.
# Formula:
#  Risk-Neutral probability
<ul>
<li> S: Current price of the underlying asset.</li>
<li>K: Strike price of the option.</li>
<li>T: Time to maturity (in years).</li>
<li>r: Risk-free interest rate.</li>
<li> σ: Volatility of the underlying asset.</li>
    <li>N: Number of time steps in the binomial tree.</Li>
</ul>
   
<p>Let \( S_0 \) be the current stock price, \( S_u \) the stock price in the up state, and \( S_d \) the stock price in the down state. The risk-neutral probability \( p \) of the up state is given by:</p>

<p>\[ p = \frac{S_0 - S_d}{S_u - S_d} \]</p>
<p>
where:
- \( S_u \) = Stock price if the market goes up
- \( S_d \) = Stock price if the market goes down
- \( S_0 \) = Current stock price</p>

# Backward Induction

<p>\[C_t = \frac{1}{1 + r} \left( p \cdot C_{u} + (1 - p) \cdot C_{d} \right)\]
</p>
</body>

In [14]:
class BiT:
    def __init__(self, last_price, strike_price, Time, risk_free, volatility, N):
        self.S = last_price
        self.K = strike_price
        self.T = Time 
        self.r = risk_free 
        self.sigma = volatility 
        self.N = N 
        self.probability_factor()
        
    def probability_factor(self):
        self.dt = self.T / self.N  # Time step
        self.u = np.exp(self.sigma * np.sqrt(self.dt))  # Up factor
        self.d = 1 / self.u  # Down factor
        self.p = (np.exp(self.r * self.dt) - self.d) / (self.u - self.d)  # Risk-neutral probability

    def option(self,option_type):
        
        asset_prices = np.zeros((self.N + 1, self.N + 1))
        for i in range(self.N + 1):
            for j in range(i + 1):
                asset_prices[j, i] = self.S * (self.u ** (i - j)) * (self.d ** j)
                
        option_values = np.zeros((self.N + 1, self.N + 1))
        for j in range(self.N + 1):
            if option_type=="put": 
                option_values[j, self.N] = max(0, self.K - asset_prices[j, self.N]) 
            elif option_type=="call":
                option_values[j, self.N] = max(0, asset_prices[j, self.N] - self.K)
            else:
                raise Exception("only two option are aviaable eithier `put` or `call`" )
                

        # Backward induction
        for i in range(self.N - 1, -1, -1):
            for j in range(i + 1):
                option_values[j, i] = np.exp(-self.r * self.dt) * (self.p * option_values[j, i + 1] + (1 - self.p) * option_values[j + 1, i + 1])

        option_price = option_values[0, 0]
        print(f"The call option price is: {option_price:.2f}")
        return option_values



In [17]:
last_price= data[data.Date.dt.year==2010]['Close'].iloc[-1]
N=6
risk_free=0.05
Time=1
strike_price=20
bit = BiT(last_price, strike_price, Time, risk_free, volatility, N)
values=bit.option("call")
print(last_price)
print(values)

The call option price is: 125.03
144.05
[[125.02541151 156.20065201 194.19057696 240.4778276  296.86755864
  365.55759298 449.22383681]
 [  0.          99.13002686 124.70567799 155.87824291 193.8654699
  240.14999999 296.53698771]
 [  0.           0.          77.83137433  98.80761776 124.38057092
  155.5504153  193.53489897]
 [  0.           0.           0.          60.30794585  77.50626726
   98.47979015 124.05      ]
 [  0.           0.           0.           0.          45.88500312
   59.98011824  77.17569634]
 [  0.           0.           0.           0.           0.
   34.00835547  45.5544322 ]
 [  0.           0.           0.           0.           0.
    0.          24.22282261]]


In [18]:
# print(pd.DataFrame(values))#.to_csv("bit.csv"))
result=pd.DataFrame(values)
result

,0,1,2,3,4,5,6
0,125.025412,156.200652,194.190577,240.477828,296.867559,365.557593,449.223837
1,0.000000,99.130027,124.705678,155.878243,193.865470,240.150000,296.536988
2,0.000000,0.000000,77.831374,98.807618,124.380571,155.550415,193.534899
3,0.000000,0.000000,0.000000,60.307946,77.506267,98.479790,124.050000
4,0.000000,0.000000,0.000000,0.000000,45.885003,59.980118,77.175696
5,0.000000,0.000000,0.000000,0.000000,0.000000,34.008355,45.554432
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.222823


In [ ]:
result.transpose()

In [ ]:
arr=[]
for i in values.transpose():
    level=0
    arr.append([])
    for j in i:
        if j!=0:
           arr[level].append(int(j))
        level+=1


# Binary tree

In [ ]:
class Tree:
    def __init__(self, root):
        self.root = root
        self.left = None
        self.right = None

def make_tree(arr):
    if not arr or not arr[0]:
        return None
    
    root = Tree(arr[0][0])
    queue = [(root, 0, 0)]
    while queue:
        current, level, index = queue.pop(0)
        
        left_index = 2 * index
        right_index = 2 * index + 1
        
        if level + 1 < len(arr):
            if left_index < len(arr[level + 1]):
                current.left = Tree(arr[level + 1][left_index])
                queue.append((current.left, level + 1, left_index))
            if right_index < len(arr[level + 1]):
                current.right = Tree(arr[level][right_index+1])
                queue.append((current.right, level + 1, right_index))
    return root
tree=make_tree(arr)
tree.right.root

# Binary tree to json file

In [ ]:
def tree_to_json(tree):
    if tree is None:
        return None
    return {
        "name": tree.root,
        "children": list(filter(None, [tree_to_json(tree.left), tree_to_json(tree.right)]))
    }
json_tree=tree_to_json(tree)
import json
with open('tree.json', 'w') as json_file:
    json.dump(json_tree, json_file, indent=4)

final output [tree visualization](https://codepen.io/ertlesil-the-bold/full/poXzPEK) .

In [19]:
import IPython as f
src="https://codepen.io/ertlesil-the-bold/full/poXzPEK"
f.display.IFrame(src,width=1000,height=1500)